Haal alle bestanden op die verwerkt moeten worden

In [12]:
from os import listdir
from os.path import isfile, join
from itertools import groupby
import re, pathlib, pandas

#folder = pathlib.Path(r'C:\Users\Caspar\Documents\Prive')
folder = pathlib.Path.cwd()

files = [f for f in sorted(listdir(folder)) if isfile(join(folder,f)) and re.match(r'sub-[0-9]{2}-run[0-9]-Putamen_19.txt',f)]
print(files)

['sub-01-run1-Putamen_19.txt', 'sub-01-run2-Putamen_19.txt', 'sub-01-run3-Putamen_19.txt', 'sub-02-run1-Putamen_19.txt', 'sub-02-run2-Putamen_19.txt', 'sub-02-run3-Putamen_19.txt']


Nu hebben we een lijst van bestanden, hier moeten we per subject, en dan per run iets uit gaan lezen, daarvoor gaan we eerst de lijst van bestanden in groepen opdelen

In [13]:
#testje
firstFile = files[1]
print(firstFile[0:6])
#eerst groeperen we de lijst van bestanden per subject, lijkt lastig maar wat het doet is:
#groeperen op de eerste 6 karakters van een bestandsnaam dit doet de groupby functie
#dat levert twee dingen op, een key en een groep. De key is de eerste 6 karakters van een bestand (bv sub-01)
#de group is de bestandsnamen die horen bij deze key (bv. ['sub-01-run1-Putamen_19.txt', 'sub-01-run2-Putamen_19.txt', 'sub-01-run3-Putamen_19.txt'])
#nu slaan we in een nieuwe lijst deze keys en bijbehorende bestanden op
groupedFiles=[[key[4:],list(group)] for key,group in groupby(files , lambda s : s[0:6])]
print(groupedFiles)

sub-01
[['01', ['sub-01-run1-Putamen_19.txt', 'sub-01-run2-Putamen_19.txt', 'sub-01-run3-Putamen_19.txt']], ['02', ['sub-02-run1-Putamen_19.txt', 'sub-02-run2-Putamen_19.txt', 'sub-02-run3-Putamen_19.txt']]]


Nu we een gegroepeerde lijst hebben kunnen we per subject de data gaan ophalen. We beginnen met een test/voorbeeld van het eerste subject

In [143]:
firstSubject = groupedFiles[0]
#print(firstSubject)
#haal de het ID en de bestanden voor het subject op
subjectID = firstSubject[0]
subjectFiles = firstSubject[1]
#print(subjectFiles)
#maak een dataframe voor het subject
dataSub1 = pandas.DataFrame([[subjectID,x] for x in range(1,1411)],columns=["SubjectID","Meetmoment"])
#print(dataSub1)

#voor elk bestand van dit subject, haal de data op
for file in subjectFiles:
    #extraheer de run uit de fileName
    run = file[7:11]
    #print(run)
    #maak de kolomnamen voor het dataframe, gebruik hiervoor run, we krijgen dan kolomnamen run1 en run1_sd, run2 en run2_sd etc
    columnNames = [run,run+"_sd"]
    #lees data van de schijf, geef het de juiste kolom namen
    data = pandas.read_csv(folder/file,names=columnNames, sep=" ",header=None, usecols=range(2))
    #print(data.head())
    #voeg de kolommen toe aan het dataframe van dit subject
    dataSub1 = dataSub1.join(data)
#het resultaat:
#print(dataSub1)

Nu we een test hebben uitgewerkt gaan we alle subjects inlezen!
De eerste (buitenste) for loop gaat voor elk subject draaien, de binnenste loop is hetzelfde als de code hierboven. Aan het einde plakken we alles aan elkaar.

In [26]:
subjectDataFrames = []
for subjectGroup in groupedFiles:
    subjectID = subjectGroup[0]
    subjectFiles = subjectGroup[1]
    #check of er wel 3 bestanden zijn voor dit subject
    assert len(subjectFiles) == 3, "Niet het juiste aantal bestanden voor subject %r. We verwachten er 3, gevonden bestanden %r" % (subjectID,subjectFiles)
    #maak een dataframe voor het subject
    dataSub = pandas.DataFrame([[subjectID,x] for x in range(1,1411)],columns=["SubjectID","Meetmoment"])
    #voor elk bestand van dit subject, haal de data op
    for file in subjectFiles:
        #extraheer de run uit de fileName
        run = file[7:11]
        #maak de kolomnamen voor het dataframe, gebruik hiervoor run, we krijgen dan kolomnamen run1 en run1_sd, run2 en run2_sd etc
        columnNames = [run,run+"_sd"]
        #lees data van de schijf, geef het de juiste kolom namen
        data = pandas.read_csv(folder/file,names=columnNames, sep=" ",header=None, usecols=range(2))
        #controleer lengte van het gelezen bestand
        assert len(data.index) == 1410, "Niet het juiste aantal rijen in %r, we vonden er %r maar verwachten er 1410" % (file,len(data.index))
        #voeg de kolommen toe aan het dataframe van dit subject
        dataSub = dataSub.join(data)
    #het resultaat:
    print(dataSub.head())
    subjectDataFrames.append(dataSub)
assert len(subjectDataFrames) == len(groupedFiles), "Het inlezen van bestanden is niet goed gegegaan, we hebben niet hetzelfde aantal dataframes (%r) als groepen bestanden (%r)" % (len(subjectDataFrames),len(groupedFiles))
allData = pandas.concat(subjectDataFrames,sort=False)
#hernoem kolommen
newColumnNames = {"run1" : "water", "run2" : "suiker", "run3" : "vet","run1_sd" : "water_sd", "run2_sd" : "suiker_sd", "run3_sd" : "vet_sd"}
allData = allData.rename(columns=newColumnNames)
#schrijf data naar bestand alles.csv
allData.to_csv(folder/"alles.csv", index=False, sep=';')

  SubjectID  Meetmoment        run1    run1_sd        run2    run2_sd  \
0        01           1  976.455825  81.313032  976.455825  81.313032   
1        01           2  981.037077  81.709665  981.037077  81.709665   
2        01           3  984.095040  82.255164  984.095040  82.255164   
3        01           4  981.028397  81.416820  981.028397  81.416820   
4        01           5  986.560267  81.004929  986.560267  81.004929   

         run3    run3_sd  
0  976.455825  81.313032  
1  981.037077  81.709665  
2  984.095040  82.255164  
3  981.028397  81.416820  
4  986.560267  81.004929  
  SubjectID  Meetmoment        run1    run1_sd        run2    run2_sd  \
0        02           1  976.455825  81.313032  976.455825  81.313032   
1        02           2  981.037077  81.709665  981.037077  81.709665   
2        02           3  984.095040  82.255164  984.095040  82.255164   
3        02           4  981.028397  81.416820  981.028397  81.416820   
4        02           5  986.56026